In [1]:
# 將 property data 與 include mailaddress 送出 (目前使用正確版本)
import os
import time
from datetime import date
import datetime
import pandas as pd
import numpy as np
import smtplib


#-------------------讀取config.ini------------------------------#  
filename = '.\config.ini'

# 配置文件读入
inifile = cp.ConfigParser()
inifile.read(filename, 'UTF-8')
uid = inifile.get("mail", "uid")
pwd = inifile.get("mail", "pwd")
smtp_ip = inifile.get("mail", "smtp_ip")
#--------------------------------------------------------------
# 读取 mail 部分 

def find_earlier_form():
    list1=['']
    path= './data_property_form'
    date_list = os.listdir(path)
    now1= date.today() 
    for i in range(60):   
        form_name = 'form({}).xls'.format(now1)
        if form_name in date_list:
            list1[0] = form_name
        else:
            now1 = now1 - datetime.timedelta(days=1)
    return(list1[0])

def send_to_mail(to_Addr, send_file_name):
    fromAddr = uid
    myPass = pwd                                   
    toAddr = to_Addr
    #
    msg = MIMEMultipart() 
    msg['From'] = fromAddr
    msg['To'] = to_Addr
    msg['Subject'] = "{} 行南嘉義中心個人保管財產清單".format(date.today())  
                                                            
    body = '''
    各位同仁:大家好，此為 {} 財產清單

    本郵件由程式自動發送，不用回信。

    附件為您所保管的財產資料，預定每月將清查乙次。

    請各位同仁留意本身所負責保管之財物，並定期更正相關資料。

    期望財產管理業務能更上軌道運作 !!!


    藍志祥敬上
    '''.format(date.today()) 
    # add text 
    msg.attach(MIMEText(body, 'plain'))

    # attach a file
    part = MIMEApplication(open(send_file_name,'rb').read()) 
    part.add_header('Content-Disposition', 'attachment', filename=send_file_name) 
    msg.attach(part) 

    server = smtplib.SMTP(smtp_ip, 25)
    server.login(fromAddr, myPass)
    server.send_message(msg)
    server.quit()


#---------------讀取資料檔-----------------
# include mailaddress
file_name = "mail_address.csv"
file_path = "./mail_address/{}".format(file_name)
csv_df = pd.read_csv(file_path)
csv_df = csv_df[csv_df['姓氏'].str.contains("2-")]
csv_df = csv_df.dropna(axis=1)
csv_df = csv_df[['姓氏','電子郵件地址']]
csv_df['姓氏'] = csv_df['姓氏'].str.replace('2-','').str.replace('工程師','').str.replace('股長','')

#---------------讀取資料檔---------------------
# include property data
file_name = find_earlier_form() 
file_path = "./data_property_form/{}".format(file_name)
xls_df = pd.read_excel(file_path)
values = {'異動者':'未指配'}
xls_df = xls_df.fillna(value=values)
xls_df['異動者'] =  xls_df['異動者'].str.strip()

# merge two dataframe
xls_df = pd.merge(xls_df, csv_df, how='inner', left_on="異動者", right_on="姓氏")
owners = list(xls_df['異動者'].unique())
owners_address = list(xls_df['電子郵件地址'].unique())
xls_df = xls_df.drop(labels = ["姓氏", "電子郵件地址"], axis=1)

# 將所屬資料 mail_out
#file_names = ["property({}).xlsx".format(address) for address in owners_address]
file_names = ["property({}).xlsx".format(owners1) for owners1 in owners]
file_paths = ["{}".format(filename) for filename in file_names]

for name, path, mail in zip(owners,file_paths,owners_address):
    xls_df[xls_df['異動者'] == name].to_excel(path,sheet_name = 'Sheet1',index=False) 
    send_to_mail(mail, path)

for remove_file in file_names:
    source = r'./{}'.format(remove_file)
    destination = r'./results(email)/{}'.format(remove_file) 
    shutil.move(source,destination)

WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
